In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./samples/MNIST_data', one_hot=True)

Extracting ./samples/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./samples/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

소프트맥스 회귀 모델 만들기

이 장에서 우리는 단일 선형 레이어로 된 소프트맥스 회귀 모델을 만들 것입니다. 다음 장에서, 우리는 이 모델을 다중 레이어 합성곱 네트워크로 된 소프트맥스 회귀로 확장할 것입니다.



In [5]:
#Placeholders: 입력 이미지와 목표 출력 클래스를 위한 노드를 만들어서 계산 그래프를 만들자
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

#Variables

W = tf.Variable(tf.zeros([784,10])) #W는 784*10 행렬
b = tf.Variable(tf.zeros([10])) #b는 10차원 백터

sess.run(tf.initialize_all_variables())#항상 세션으로 사용하기 전에 초기화

예측 클래스 및 비용 함수(Cost Function)

이제 우리 회귀 모델을 만들 수 있습니다. 딱 한 줄만 필요합니다! 벡터화된 입력 이미지 x를 가중치 행렬 W로 곱하고, 편향(bias) b를 더한 다음 각 클래스에 지정된 소프트맥스 확률들을 계산합니다.

In [8]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

#훈련 과정에소 최소화 될 비용 함수들을 지정. 비용함수는 목표와 모델 예측간의
#교차 엔트로피

cross_entropy = -tf.reduce_sum(y * tf.log(y))

모델 훈련시키기

이제 모델과 훈련 비용 함수를 정의했으므로, TensorFlow를 사용하여 모델을 훈련하는 것은 간단합니다. TensorFlow는 전체 계산 그래프를 알고 있으므로, 각 변수들에 대하여 비용 그라디언트를 찾기 위해 자동 미분을 사용할 수 있습니다. TensorFlow에는 다양한 내장 최적화 알고리즘들 이 있습니다. 이 예제에서는 교차 엔트로피를 하강시키기 위해 가파른 그라디언트 하강법 (steepest gradient descent) 을 0.01의 보폭으로 사용할 것입니다.

In [16]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
#이 한 줄에서 TensorFlow가 진짜로 하는 일은 
#계산 그래프에 새 작업을 추가하는 일입니다. 
#이러한 작업들에는 그라디언트를 계산하고, 매개 변수가 
#갱신 보폭을 계산하고, 갱신된 보폭을 매개 변수에 반영하는 단계를 포함하고 있습니다.

for i in range(1000):
    batch = mnist.train.next_batch(50) #50개 훈련 예제 불러오기
    train_step.run(feed_dict = {x: batch[0], y_: batch[1]})

#feed dict를 활용하면 계산 그래프 내의 어떠한 텐서도 대체 가능

모델 평가하기

우리 모델이 얼마나 좋을까요?

첫번째로 우리가 맞는 라벨을 예측했는지를 확인할 것입니다.. tf.argmax는 특정한 축을 따라 가장 큰 원소의 색인을 알려주는 엄청나게 유용한 함수입니다. 예를 들어 tf.argmax(y,1) 는 진짜 라벨이 tf.argmax(y_,1) 일때 우리 모델이 각 입력에 대하여 가장 정확하다고 생각하는 라벨입니다. 우리는 tf.equal 을 이용해 예측이 실제와 맞았는지 확인할 수 있습니다.

In [17]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

#결과는 부울 리스트를 주고 부정 소솟점으로 캐스팅한 후 평균값 구하기
#예를 들어, [True, False, True, True]는 [1,0,1,1] 이 되고 평균값은 0.75

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.098


## 다중 레이어 합성곱 네트워크


가중치 초기화

이 모델을 만들기 위해서, 우리는 많은 가중치 및 편향을 만들 필요가 있습니다. 일반적으로 작은 크기의 노이즈로 가중치를 초기화해서 대칭성을 파괴하고, 그라디언트가 0이 되는 경우를 방지합니다. 우리가 ReLU 뉴런을 사용할 것이므로, 그것들을 약간 양수의 편향을 주어 초기화해서 "죽은 뉴런" 이 되는 것을 막는 것도 좋은 시도입니다. 모델을 만들 때 마다 이 과정을 반복하는 대신, 우리를 위해 이걸 해줄 편리한 두 함수를 만듭시다.

In [18]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

합성곱과 풀링 (Pooling)

TensorFlow는 또한 합성곱과 풀링 작업을 위한 많은 유연성을 제공합니다. 어떻게 경계를 다룰까요? 이동 (stride) 크기는 얼마인가요? 이 예시에서, 우리는 언제나 가장 기본적인 버전을 선택할 것입니다. 합성곱은 하나의 이동 크기를 사용하고, 0로 패딩되어 결과적으로 출력의 크기가 입력의 크기와 같게 됩니다. 풀링은 2x2 블럭의 평범한 max pooling 입니다. 코드를 깨끗하게 유지하기 위해 이러한 작업들을 함수로 추상화합시다.

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

